In [ ]:
import os#
#os.chdir('c:\\Users\\ma11115\\OneDrive - Imperial College London\\github_folder\\STK_SEARCH\\')
os.chdir("/rds/general/user/ma11115/home/STK_Search/STK_search")
from pathlib import Path#
import pandas as pd
import torch
import numpy as np
import importlib
from stk_search.utils import database_utils

os.makedirs('data/output/search_experiment/search_exp_database', exist_ok=True)

: 

In [ ]:
# run a search experiment with the new target
from stk_search.utils import run_hpc_utils
importlib.reload(run_hpc_utils)
df_total_path = "data/output/Full_dataset/New_target07052024.csv"
df_precursor_path ="data/output/Prescursor_data/precursor_data_200324.pkl"
df_precursor_Mordred_path="/rds/general/user/ma11115/home/STK_Search/STK_search/data/output/Prescursor_data/precursor_data_mordred_descriptor_200324.pkl"

search_space_loc = "data/input/EXP_search_space/6_frag_all_2024_05_11_22_40_37//search_space_0.pkl"

: 

In [ ]:
# load and check the search space
from stk_search import Search_space
search_space = pd.read_pickle(search_space_loc)
print(' Search space condition list:', search_space.conditions_list)
print('shape of df_precursors:', search_space.df_precursors.shape)
print('search space syntax:', search_space.syntax)


: 

In [ ]:
# load and check the dataframe to search over and the df_precursors
df_Benchmark , df_precursors= database_utils.load_data_from_file(df_total_path, df_precursor_path,add_feature_frag=True)
df_precursor_Mordred = pd.read_pickle(df_precursor_Mordred_path)
print('shape of df_Benchmark:', df_Benchmark.shape)
print('shape of df_precursors:', df_precursors.shape)
print('shape of df_precursor_Mordred:', df_precursor_Mordred.shape)

: 

In [ ]:
num_of_elem_in_SP = search_space.check_df_for_element_from_SP(df_Benchmark).shape
print('number of elements in the benchmark in the search space:', num_of_elem_in_SP)

: 

In [ ]:
# set the benchmark parameters  
exp_name = "Benchmark_75K_dataset"
num_elem_initialisation=50
num_iteration=500
target = 'target'
aim = 'maximise'
case_name_list = ['evolution_algorithm','random','BO_precursor',"ea_surrogate", "BO_learned","ea_surrogate_new", "BO_learned_new"]
config_dir_list = ["/rds/general/user/ma11115/home/STK_Search/STK_search/data/representation_learning/6-frag/target/240322/SchNet/splitrand-nummol50000"]
config_dir_list_new = ["/rds/general/user/ma11115/home/STK_Search/STK_search/data/representation_learning/6-frag/target/240412/SchNet/splitrand-nummol68864"]

: 

In [ ]:
# check the representation generated using the learned representation ( using a gnn for each fragment)
config_dir = config_dir_list[0]
from src.dev_scripts.run_search_new import load_representation_BO_graph_frag
representation = load_representation_BO_graph_frag(config_dir,df_total_path)
representation.generate_repr(df_Benchmark[[f'InChIKey_{i}' for i in range(6)]].loc[0:100,:])

: 

In [ ]:
# check the representation generated using the learned representation building the polymer from fragment for 
config_dir = config_dir_list_new[0]
from src.dev_scripts.run_search_new import load_representation_model
representation, pymodel = load_representation_model(config_dir)
representation.generate_repr(df_Benchmark[[f'InChIKey_{i}' for i in range(6)]].loc[0:100,:])

: 

: 

In [ ]:


def run_search_on_hpc(
    case_name="ea_surrogate",
    df_path_total=df_total_path,
    df_precursors_path=df_precursor_path,
    frag_properties="selected",
    config_dir="",
    dataset_representation_path = None,
    test=False,
    lim_counter = 5,
    which_acquisition = "EI"
):
    string_to_run_notebook, script_qsub = run_hpc_utils.generate_string_run(
        case=case_name,
        target=target,
        aim=aim,
        num_elem_initialisation=num_elem_initialisation,
        num_iteration=num_iteration,
        df_path=df_path_total,
        benchmark=True,
        test_name=exp_name,
        df_precursors_path=df_precursors_path,
        frag_properties=frag_properties,
        config_dir=config_dir,
        dataset_representation_path=dataset_representation_path,
        lim_counter = lim_counter,
        which_acquisition = which_acquisition
    )
    if test:
        %run {string_to_run_notebook}
        return string_to_run_notebook
    run_hpc_utils.submit_job(script_qsub, case_name)


for case_name in case_name_list[4:]:
    if case_name == 'ea_surrogate' or case_name == "BO_learned":
        for config_dir in config_dir_list:
            run_search_on_hpc(
                case_name=case_name,
                df_precursors_path=df_precursor_path,
                frag_properties="selected",
                config_dir = config_dir,
            )
    elif case_name == 'ea_surrogate_new' or case_name == "BO_learned_new":
        for config_dir in config_dir_list_new:
            run_search_on_hpc(
                case_name=case_name,
                df_precursors_path=df_precursor_path,
                frag_properties="selected",
                config_dir = config_dir,
            )
    else:
        run_search_on_hpc(
            case_name=case_name,
            df_precursors_path=df_precursor_path,
            frag_properties="selected",
        )
        if case_name =='BO_precursor':
            run_search_on_hpc(
                case_name=case_name,
                df_precursors_path=df_precursor_Mordred_path,
                frag_properties="all",
        )

src/dev_scripts/run_search_new.py  --case evolution_algorithm --test_name Benchmark_75K_dataset --target target --aim maximise --benchmark 1 --num_iteration 500 --num_elem_initialisation 50 --which_acquisition EI --dataset_representation_path None --df_path data/output/Full_dataset/New_target07052024.csv --df_precursors_path data/output/Prescursor_data/precursor_data_200324.pkl --search_space_loc /rds/general/user/ma11115/home/STK_Search/STK_search/data/input/search_space/test/search_space1.pkl --frag_properties selected --lim_counter 5
9419294[].pbs
src/dev_scripts/run_search_new.py  --case random --test_name Benchmark_75K_dataset --target target --aim maximise --benchmark 1 --num_iteration 500 --num_elem_initialisation 50 --which_acquisition EI --dataset_representation_path None --df_path data/output/Full_dataset/New_target07052024.csv --df_precursors_path data/output/Prescursor_data/precursor_data_200324.pkl --search_space_loc /rds/general/user/ma11115/home/STK_Search/STK_search/dat

/rds/general/user/ma11115/home/STK_Search/STK_search/src/dev_scripts/run_search_new.py:50: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_total = pd.read_csv(df_path)


ea_surrogate   case  
config loaded from /rds/general/user/ma11115/home/STK_Search/STK_search/data/representation_learning/6-frag/target/240322/SchNet/splitrand-nummol50000
cpu
Model loaded:  /rds/general/user/ma11115/home/STK_Search/STK_search/data/representation_learning/6-frag/target/240322/SchNet/splitrand-nummol50000/epoch=18-val_loss=0.21-other_metric=0.00.ckpt
lr 0.0005
loading model from checkpoint /rds/general/user/ma11115/home/STK_Search/STK_search/data/representation_learning/6-frag/target/240322//SchNet/splitrand-nummol50000//transformer/epoch=43-val_loss=0.42-other_metric=0.00.ckpt
Inputs saved.
element Inchikey suggested: HGEKDCOIJBNPOS-UHFFFAOYSA-N, Eval: -3.6235164033831895
element Inchikey suggested: GSIABESMLPWKQW-UHFFFAOYSA-N, Eval: 0.1753025939554899
element Inchikey suggested: HUKBNKNJERLFGK-HDCDBCLTSA-N, Eval: -1.6288530348398922
element Inchikey suggested: ZLTHXODSYHPARP-UHFFFAOYSA-N, Eval: -3.8332807473813815
element Inchikey suggested: QOMUCTKITJDKTA-UHFFFAOYSA

X_unsqueezed shape is  torch.Size([719, 128])
max acquisition value is  -0.16547573
new_element_df shape is  (719, 6)
368
element Inchikey suggested: BQUVWSNWVYERNM-UHFFFAOYSA-N, Eval: -0.0280852222266346
iteration 317 completed
max fitness acquired: 0.827915671434394
min fitness acquired: -3.857297499232713
new fitness acquired: -0.0280852222266346
X_unsqueezed shape is  torch.Size([534, 128])
max acquisition value is  -0.16547573
new_element_df shape is  (534, 6)
319
element Inchikey suggested: WSCXWFLBEQUXEP-GGPBTZDJSA-N, Eval: 0.1757588686778715
iteration 318 completed
max fitness acquired: 0.827915671434394
min fitness acquired: -3.857297499232713
new fitness acquired: 0.1757588686778715
X_unsqueezed shape is  torch.Size([609, 128])
max acquisition value is  -0.16547573
new_element_df shape is  (609, 6)
418
element Inchikey suggested: CPXYDUBGZPXABQ-UHFFFAOYSA-N, Eval: 0.4696823065103349
iteration 319 completed
max fitness acquired: 0.827915671434394
min fitness acquired: -3.85729

: 